In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Boas práticas para a criação de prompts

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Execute no Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      Veja no GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Execute no Vertex AI Workbench
    </a>
  </td>
</table>

## Visão geral

Este notebook aborda os fundamentos de design de prompts, incluindo algumas práticas recomendadas.

Saiba mais sobre o design de prompt na [documentação oficial](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

### Objetivo

Neste notebook, você aprenderá as práticas recomendadas sobre o design de prompts -- como projetar prompts para melhorar a qualidade de suas respostas.

Este notebook abrange as seguintes práticas recomendadas para engenharia imediata:

- Ser conciso
- Seja específico e com um texto bem definido
- Peça uma tarefa de cada vez
- Transforme tarefas generativas em tarefas de classificação
- Melhore a qualidade da resposta incluindo exemplos

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Studio

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

## Primeiros Passos

### Instalando os SDK da Vertex AI

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user

**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [ ]:
# # Reinicia automaticamente o kernel após as instalações para que seu ambiente possa acessar os novos pacotes
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../setup-env/README.md).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [ ]:
# import vertexai

# PROJECT_ID = "[seu-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

from vertexai.language_models import TextGenerationModel

#### Carregando o modelo `text-bison`

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## Boas práticas de design de prompts

A engenharia de prompt trata de como projetar seus prompts para que a resposta seja o que você realmente esperava ver.

A ideia de usar prompts "desagradáveis" é minimizar o ruído em seu prompt para reduzir a possibilidade de o LLM interpretar mal a intenção do prompt. Abaixo estão algumas diretrizes sobre como projetar prompts "desagradáveis".

Nesta seção, você abordará as seguintes práticas recomendadas quando a engenharia solicitar:

* Ser conciso
* Seja específico e com texto bem definido
* Peça uma tarefa de cada vez
* Melhore a qualidade da resposta incluindo exemplos
* Transforme tarefas generativas em tarefas de classificação para melhorar a segurança

### Seja conciso

🛑 Não recomendado. O prompt abaixo é desnecessariamente detalhado.

In [ ]:
prompt = "Que nomes você acha que seriam interessantes para uma floricultura que se especializa mais em buquês de flores secas do que frescas? Muitíssimo brigado!"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

✅ Recomendado. O prompt abaixo é direto ao ponto e conciso.

In [ ]:
prompt = "Sugira cinco nomes para uma floricultura que vende buquês de flores secas"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

### Seja específico e escreva textos bem definidos

Suponha que você queira pensar em maneiras criativas de descrever a Terra.

🛑 Não recomendado. O prompt abaixo é muito genérico.

In [ ]:
prompt = "Fale-me sobre a Terra"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

✅ Recomendado. O prompt abaixo é específico e bem definido.

In [ ]:
prompt = "Gere uma lista de motivos que fazem a Terra ser única comparada à outros planetas"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

### Peça uma tarefa de cada vez

🛑 Não recomendado. O prompt abaixo tem duas partes para a pergunta que pode ser feita separadamente.

In [ ]:
prompt = "Qual é a melhor forma para ferver água e por que o céu é azul?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

✅ Recomendado. Os prompts abaixo solicitam uma tarefa por vez.

In [ ]:
prompt = "Qual é a melhor forma para ferver água?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

In [ ]:
prompt = "Por que o céu é azul?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

### Cuidado com as alucinações

Embora os LLMs tenham sido treinados em uma grande quantidade de dados, eles podem gerar textos contendo declarações não fundamentadas na verdade ou na realidade; essas respostas do LLM são frequentemente chamadas de "alucinações" devido às suas capacidades limitadas de memorização. Observe que simplesmente solicitar que o LLM forneça uma citação não é uma solução para esse problema, pois há instâncias de LLMs que fornecem citações falsas ou imprecisas. 

Lidar com alucinações é um desafio fundamental dos LLMs e uma área de pesquisa em andamento, por isso é importante estar ciente de que os LLMs podem parecer dar a você declarações confiantes e corretas que, na verdade, são incorretas.

Observe que, se você pretende usar LLMs para os casos de uso criativo, alucinar pode ser bastante útil.

Tente o prompt como o abaixo repetidamente. Você pode notar que às vezes ele dirá com confiança, mas imprecisamente, "O primeiro elefante a visitar a lua foi Luna".

In [ ]:
prompt = "Quem foi o primeiro elefante a visitar a lua?"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

### Transforme tarefas generativas em tarefas de classificação para reduzir a variabilidade de saída

#### Tarefas generativas levam a uma maior variabilidade de saída

O prompt abaixo resulta em uma resposta aberta, útil para brainstorming, mas a resposta é altamente variável.

In [ ]:
prompt = "Sou um estudante de ensino médio. Recomende-me atividades de programação para melhorar meus skills"

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

#### Tarefas de classificação reduzem a variabilidade de saída

O prompt abaixo resulta em uma escolha e pode ser útil se você quiser que a saída seja mais fácil de controlar.

In [ ]:
prompt = """Sou um estudante de ensino médio. Quais destas atividades você recomenda e porquê:
a) Aprender Python
b) Aprender Javascript
c) Aprender Fortran
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

### Melhore a qualidade da resposta incluindo exemplos

Outra maneira de melhorar a qualidade da resposta é adicionar exemplos em seu prompt. O LLM aprende no contexto dos exemplos sobre como responder. Normalmente, um a cinco exemplos (shots) são suficientes para melhorar a qualidade das respostas. Incluir muitos exemplos pode fazer com que o modelo ajuste demais os dados e reduza a qualidade das respostas.

Semelhante ao treinamento de modelo clássico, a qualidade e a distribuição dos exemplos são muito importantes. Escolha exemplos representativos dos cenários que você precisa que o modelo aprenda e mantenha a distribuição dos exemplos (por exemplo, número de exemplos por classe no caso de classificação) alinhada com sua distribuição real.

#### Zero-shot prompt

Abaixo está um exemplo de prompt zero-shot, onde você não fornece nenhum exemplo para o LLM dentro do próprio prompt.

In [ ]:
prompt = """Decida se um Tweet apresenta um sentimento positivo, negativo ou neutro.

Tweet: Eu amei os seus videos mais recentes no YouTube!
Sentimento: 
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

#### One-shot prompt

Abaixo está um exemplo one-shot, onde você fornece um exemplo para o LLM dentro do prompt para fornecer alguma orientação sobre o tipo de resposta que você deseja.

In [ ]:
prompt = """Decida se um Tweet apresenta um sentimento positivo, negativo ou neutro.

Tweet: Eu amei os seus videos mais recentes no YouTube!
Sentimento: positivo

Tweet: Foi difícil. Super chato 😠
Sentimento:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

#### Few-shot prompt

Abaixo está um exemplo de few-shot, onde você fornece um exemplo para o LLM dentro do prompt para dar alguma orientação sobre o tipo de resposta que você deseja.

In [ ]:
prompt = """Decida se um Tweet apresenta um sentimento positivo, negativo ou neutro.

Tweet: Eu amei os seus videos mais recentes no YouTube!
Sentimento: positivo

Tweet: Foi difícil. Super chato 😠
Sentimento: negativo

Tweet: Uma coisa me surpreendeu neste video - Ele foi realmente original. Ele não foi o mesmo velho conteúdo reciclado. Assista - você não se arrependerá.
Sentimento:
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

#### Escolhendo entre os métodos zero-shot, one-shot ou few-shot

Qual técnica de prompt usar dependerá exclusivamente do seu objetivo. Os prompts zero-shot são mais abertos e podem fornecer respostas criativas, enquanto os prompts one-shot e few-shot ensinam o modelo a se comportar para que você possa obter respostas mais previsíveis que sejam consistentes com os exemplos fornecidos.